In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import transforms
from model_functions import *
from scf import *
from data_loader_scf import *
from model_types import efficientnet_1, custom_model1, efficientnet_2
from torchsummary import summary
from torchvision import models
from copy import deepcopy

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
effnet_model = efficientnet_2.Model(in_channels=4)
effnet_model.unfreeze_params()

In [ ]:
state_dict_path = r'C:\AAA\FYP\mimo-radar-drone-detection-fyp\final code\saved_models_type\efficientnet_class\from_epoch_60\efficientnet_unfreezed_60_50_epoch_76.pt'
state_dict = torch.load(state_dict_path)

effnet_model.load_state_dict(state_dict['Model state'])
effnet_model = effnet_model.to(device)

optimizer = optim.Adam(effnet_model.parameters(), lr=0.00001, weight_decay=1e-5)
optimizer.load_state_dict(state_dict['Optimizer state'])
# param_group = optimizer.param_groups[0]
# param_group['lr'] = 1e-5
# param_group['weight_decay'] = 5e-4
optimizer

In [ ]:
train_dir = '../../Data/Dataset/Train/scf/'
test_dir = '../../Data/Dataset/Test/scf/'

train_data = data_loader(train_dir, shuffle=True, batch_size=128, binary=True)
val_data = data_loader(test_dir, shuffle=False, batch_size=128, binary=True)

In [ ]:
summary(deepcopy(effnet_model).to(device), (1,512,512))

In [ ]:
accuracy = evaluate_model_binary(effnet_model, val_data, device)

In [ ]:
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([0.282]).to(device))

In [ ]:
epochs = 10
loss, optimizer = train_model_binary(effnet_model, epochs, criterion, optimizer, train_data, device)

In [ ]:
accuracy = evaluate_model_binary(effnet_model, val_data, device)

In [ ]:
save_path = r'C:\AAA\FYP\mimo-radar-drone-detection-fyp\final code\saved_models_type\efficientnet_clipped\from_epoch_40'
save_model(save_path, effnet_model, f'efficientnet_unfreezed_40_10_epoch_{round(accuracy)}', optimizer=optimizer)

In [ ]:
# from data_loader_scf_visualization import *

# validation_data = data_loader_visualize(test_dir, shuffle=False, batch_size=1, binary=True)A

In [ ]:
# from sklearn.metrics import confusion_matrix
# import seaborn as sn
# import pandas as pd
# import numpy as np

# y_pred = []
# y_true = []

# effnet_model.eval()
# effnet_model = effnet_model.to(device)

# df_fp = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[]}
# df_fn = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[]}

# with torch.no_grad():
#     for inputs, labels, label_name, class_label in validation_data:
#         inputs = inputs.to(dtype=torch.float)
#         inputs, labels = inputs.to(device), labels.to(device).reshape(-1,1)
#         output = effnet_model(inputs)
#         predicted = (output > 0).int().reshape(-1).data.cpu().numpy()
#         y_pred.extend(predicted)

#         labels = labels.reshape(-1).data.cpu().numpy()
#         y_true.extend(labels)
        
#         if predicted[0] == labels[0]:
#             pass
#         elif predicted[0] == 0:
#             df_fn[int(class_label[0])].append(label_name[0])
#         else:
#             df_fp[int(class_label[0])].append(label_name[0])

# classes = ('reference', 'drone')

# cf_matrix = confusion_matrix(y_true, y_pred)
# df_cm = pd.DataFrame(cf_matrix, index = [i for i in classes],
#                      columns = [i for i in classes])
# plt.figure(figsize = (12,7))
# sn.heatmap(df_cm, annot=True)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np

y_pred = []
y_true = []

effnet_model = effnet_model.to(device)
effnet_model.eval()

with torch.no_grad():
    for inputs, labels in val_data:
        inputs = inputs.to(dtype=torch.float)
        inputs, labels = inputs.to(device), labels.to(device).reshape(-1,1)
        output = effnet_model(inputs)
        predicted = (output > 0).float().reshape(-1).data.cpu().numpy()
        y_pred.extend(predicted)
        
        labels = labels.reshape(-1).data.cpu().numpy()
        y_true.extend(labels)

classes = ('reference', 'drone')

cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)

In [ ]:
epochs = 10
loss, optimizer = train_model_binary(effnet_model, epochs, criterion, optimizer, train_data, device)

accuracy = evaluate_model_binary(effnet_model, val_data, device)

save_model(save_path, effnet_model, f'efficientnet_unfreezed_40_20_epoch_{round(accuracy)}', optimizer=optimizer)

from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np

y_pred = []
y_true = []

efnet_model = effnet_model.to(device)
efnet_model.eval()

with torch.no_grad():
    for inputs, labels in val_data:
        inputs = inputs.to(dtype=torch.float)
        inputs, labels = inputs.to(device), labels.to(device).reshape(-1,1)
        output = efnet_model(inputs)
        predicted = (output > 0).float().reshape(-1).data.cpu().numpy()
        y_pred.extend(predicted)
        
        labels = labels.reshape(-1).data.cpu().numpy()
        y_true.extend(labels)

classes = ('reference', 'drone')

cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)